In [1]:
import sys
import wget
import newspaper
from newspaper import Article
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
import collections
from urllib import request
from collections import OrderedDict
from urllib.error import HTTPError, URLError
from urllib.parse import urlparse
import sys
import ast
import time
from pandarallel import pandarallel
import requests
import datefinder
from datetime import datetime
from tqdm import tqdm
import psutil
from selenium import webdriver
from multiprocessing import Pool
import random
from pathlib import Path

In [2]:
country = ""
news_outlet = ""
extract_text_sleep = random.randint(2,4)
date_format = "%Y-%m-%d"

In [3]:
filepaths = ['links_to_be_processed/Wooseong/links_spain_lasprovincias.csv','links_to_be_processed/Wooseong/links_spain_abcdesevilla.csv']
# filepaths = []

# for path in Path('./links_to_be_processed/Wooseong').rglob('*.csv'):
#     if 'unprocessed' not in str(path):
#         filepaths.append(str(path))

print(filepaths)

['links_to_be_processed/Wooseong/links_spain_lasprovincias.csv', 'links_to_be_processed/Wooseong/links_spain_abcdesevilla.csv']


In [4]:
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [5]:
def extract_text(url):
    global country, news_outlet
    text, title, date = None, None, None
    try:
        result = [country, news_outlet] + [''] * 3 
        article = Article(url, keep_article_html=False)
        article.download()
        time.sleep(extract_text_sleep)
        article.parse()
        text = article.text
        
        if text:
            text_copy = text
            title = article.title
            date = article.publish_date
            if date:
                date = date.strftime(date_format)
            else:
                matches = datefinder.find_dates(text_copy)
                most_recent_datetime = sorted(matches)[-1]
                date = most_recent_datetime.strftime(date_format)
    finally:
      if title:
        result[2] = title
      if date:
        result[3] = date
      if text:
        result[4] = text
      return result 


def extract_text_from_df(df):
    content = df['link'].map(lambda x: extract_text(x))
    df['content'] = content
    df[['country', 'news_outlet', 'title', 'date', 'text']] = pd.DataFrame(df.content.tolist(), index= df.index)
    df = df.drop(['content'], axis=1)
    if 'Unnamed: 0' in df:
        df = df.drop(['Unnamed: 0'], axis=1)
    return df

In [ ]:
for filepath in filepaths: 
    root,owner,filename =  filepath.split('/')
    filename = filename.replace('links_','')
    
    country, news_outlet = filename.replace(".csv","").split("_")
    output_path = root+'/'+owner+'/results'
    output_path_file = output_path + '/' + 'unprocessed_' + filename
    final_output_file = output_path +  '/' + filename
    
    if not(os.path.exists(output_path)):
        os.mkdir(output_path)
        
    df = pd.read_csv(filepath)
    
    start = 0
    if filepath == "links_to_be_processed/Wooseong/links_spain_lasprovincias.csv":
        start = 192640
    else:
        start = 0
        
    limit = 160
    number_of_cores = 40
    completed = 0
    failed_to_retrieve_unprocessed = False
    
    res = []
    while start < len(df):
        start_time = time.time()
        demo_df = df[start:start+limit].copy()
        no_of_text_retrieved = 0
        
        test_df = parallelize_dataframe(demo_df, extract_text_from_df, number_of_cores)
        no_of_text_retrieved = (test_df['text'] != '').sum()
        
            
        test_df.to_csv(output_path_file, mode='a', header=not os.path.exists(output_path_file))
        res.append(test_df)
       
        
        if completed == 0 and start != 0:
            try:
                unprocessed_csv_df = pd.read_csv(output_path_file, converters={'content': pd.eval})
                completed = unprocessed_csv_df['content'].map(lambda x: True if x[4] != '' else False).sum()
            except:
                try:
                    unprocessed_csv_df = pd.read_csv(output_path_file, on_bad_lines='skip', converters={'content': pd.eval})
                    completed = unprocessed_csv_df['content'].map(lambda x: True if x[4] != '' else False).sum()
                except:
                    failed_to_retrieve_unprocessed = True
            
        print(f'Processing links file {filepath}')
        print(f'{no_of_text_retrieved} / {len(test_df)} text retrieved')
        if not failed_to_retrieve_unprocessed:
            completed += no_of_text_retrieved
            print(f'Total text which has been successfully read till now - {round((((completed) / (start+limit))* 100), 2)}%')
        
        end_time = time.time()
        
        print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
        
        print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
        start+=limit
    
    res_df = pd.concat(res)
    if not(os.path.exists(final_output_file)):
        res_df.to_csv(final_output_file) 
    else:
        print(f"{final_output_file} already exists. Did not create new csv")


<ipython-input-6-a03576859728>:42: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_csv_df = pd.read_csv(output_path_file, converters={'content': pd.eval})


Processing links file links_to_be_processed/Wooseong/links_spain_lasprovincias.csv
160 / 160 text retrieved
Batch of data of row range 192640-192800 complete in 28.96 seconds
83.05% complete


<ipython-input-6-a03576859728>:46: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_csv_df = pd.read_csv(output_path_file, on_bad_lines='skip', converters={'content': pd.eval})
<ipython-input-6-a03576859728>:42: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_csv_df = pd.read_csv(output_path_file, converters={'content': pd.eval})


Processing links file links_to_be_processed/Wooseong/links_spain_lasprovincias.csv
160 / 160 text retrieved
Batch of data of row range 192800-192960 complete in 30.27 seconds
83.12% complete


<ipython-input-6-a03576859728>:46: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_csv_df = pd.read_csv(output_path_file, on_bad_lines='skip', converters={'content': pd.eval})
<ipython-input-6-a03576859728>:42: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_csv_df = pd.read_csv(output_path_file, converters={'content': pd.eval})


...

<ipython-input-6-a03576859728>:46: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_csv_df = pd.read_csv(output_path_file, on_bad_lines='skip', converters={'content': pd.eval})
<ipython-input-6-a03576859728>:42: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_csv_df = pd.read_csv(output_path_file, converters={'content': pd.eval})


Processing links file links_to_be_processed/Wooseong/links_spain_lasprovincias.csv
154 / 160 text retrieved
Batch of data of row range 210240-210400 complete in 33.72 seconds
90.63% complete


<ipython-input-6-a03576859728>:46: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_csv_df = pd.read_csv(output_path_file, on_bad_lines='skip', converters={'content': pd.eval})
<ipython-input-6-a03576859728>:42: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_csv_df = pd.read_csv(output_path_file, converters={'content': pd.eval})


Processing links file links_to_be_processed/Wooseong/links_spain_lasprovincias.csv
157 / 160 text retrieved
Batch of data of row range 210400-210560 complete in 33.65 seconds
90.7% complete


<ipython-input-6-a03576859728>:46: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_csv_df = pd.read_csv(output_path_file, on_bad_lines='skip', converters={'content': pd.eval})
